## Adversarial Training in Keras
This notebook is part of Module 3 of the [ODSC Europe 2021](https://odsc.com/europe/) workshop [Adversarial Attacks and Defence in Computer Vision 101](https://odsc.com/speakers/adversarial-attacks-and-defence-in-computer-vision-101/).  
![ODSC Logo](https://opendatascience.com/wp-content/uploads/2021/01/odsceutop.png)  
It shows how to perform Adversarial Training on a [DenseNet201](https://www.tensorflow.org/api_docs/python/tf/keras/applications/DenseNet201) model trained on the [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) data set.  Adversarial attacks and training are performed throgh the Open Source [ART](https://adversarial-robustness-toolbox.org/) (Adversarial Robustness Toolbox) Python library.
For faster execution is suggested to execute this notebook in a GPU or TPU runtime.

Install ART.

In [ ]:
!pip install adversarial-robustness-toolbox[tensorflow]

     |████████████████████████████████| 983kB 10.0MB/s 
     |████████████████████████████████| 3.4MB 46.6MB/s 
     |████████████████████████████████| 686kB 42.0MB/s 
     |████████████████████████████████| 25.3MB 114kB/s 
  Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


Import the necessary Python packages.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.utils import to_categorical

from art import config
from art.utils import load_dataset, get_file
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import BasicIterativeMethod
from art.defences.trainer import AdversarialTrainer

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

Load the CIFAR-10 data set.

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 4s 0us/step


Preprocess the data and convert the labels to one-hot encodings.

In [ ]:
def preprocess_data(X, Y):
    X = preprocess_input(X)
    Y = to_categorical(Y)

    return X, Y

In [ ]:
x_train, y_train = preprocess_data(x_train, y_train)
x_test, y_test = preprocess_data(x_test, y_test)

Load the pre-trained CNN model on CIFAR-10.

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
model_path = './cifar10.h5'
classifier_model = load_model(model_path)

In [ ]:
classifier = KerasClassifier(model=classifier_model, use_logits=False, clip_values=[0,255])

Evaluate the performance of the model towards test legit images.

In [ ]:
n = 100
y_pred = classifier.predict(x_test[:n])
accuracy = np.mean(np.argmax(y_pred, axis=1) == np.argmax(y_test[:n], axis=1))

In [ ]:
print("Accuracy on first %i benign test samples: %f" % (n, accuracy))

Accuracy on first 100 benign test samples: 0.900000


Generate some adversarial samples (using FGSM) and evaluate the performance of the network against them.

In [ ]:
attacker = FastGradientMethod(classifier, eps=0.5)
x_test_adv = attacker.generate(x_test[:100])

In [ ]:
x_test_adv_pred = np.argmax(classifier.predict(x_test_adv), axis=1)
nb_correct_adv_pred = np.sum(x_test_adv_pred == np.argmax(y_test[:100], axis=1))

print("Adversarial test data (first 100 images):")
print("Correctly classified: {}".format(nb_correct_adv_pred))
print("Incorrectly classified: {}".format(100-nb_correct_adv_pred))

Adversarial test data (first 100 images):
Correctly classified: 39
Incorrectly classified: 61


Perform the adversarial training.

In [ ]:
trainer = AdversarialTrainer(classifier, attacker, ratio=1.0)

In [ ]:
trainer.fit(x_train, y_train, nb_epochs=20, batch_size=50)

Evaluate the robust classifier against the legit test data.

In [ ]:
x_test_robust_pred = np.argmax(classifier.predict(x_test[:100]), axis=1)
nb_correct_robust_pred = np.sum(x_test_robust_pred == np.argmax(y_test[:100], axis=1))

print("Original test data (first 100 images):")
print("Correctly classified: {}".format(nb_correct_robust_pred))
print("Incorrectly classified: {}".format(100-nb_correct_robust_pred))

Original test data (first 100 images):
Correctly classified: 76
Incorrectly classified: 24


Evaluate the robust classifier against adversarial samples (FGSM).

In [ ]:
attacker_robust = FastGradientMethod(classifier, eps=0.5)
x_test_adv_robust = attacker_robust.generate(x_test[:100])

In [ ]:
x_test_adv_robust_pred = np.argmax(classifier.predict(x_test_adv_robust), axis=1)
nb_correct_adv_robust_pred = np.sum(x_test_adv_robust_pred == np.argmax(y_test[:100], axis=1))

print("Adversarial test data (first 100 images):")
print("Correctly classified: {}".format(nb_correct_adv_robust_pred))
print("Incorrectly classified: {}".format(100-nb_correct_adv_robust_pred))

Adversarial test data (first 100 images):
Correctly classified: 68
Incorrectly classified: 32
